In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [3]:
itp_id = 4 ## AC Transit

In [4]:
import numpy as np

In [5]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [6]:
date_range

array(['2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03',
       '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07',
       '2022-05-08'], dtype='datetime64[D]')

# Generate analysis data for all dates

In [7]:
# for date in date_range:
#     date = date.astype(dt.date)
#     try:
#         rt_analysis = rt.OperatorDayAnalysis(itp_id, date, pbar)
#         rt_analysis.export_views_gcs()
#         print(f'complete for date: {date}')
#     except:
#         print(f'failed for date: {date}')
#         continue

In [8]:
# for date in date_range:
#     date = date.astype(dt.date)
#     try:
#         rt_day = rt_filter_map_plot.from_gcs(itp_id, date)
#         print(f'complete for date: {date}')
#     except:
#         print(f'failed for date: {date}')
#         continue

In [9]:
rt_day = rt_filter_map_plot.from_gcs(4, dt.date(2022, 5, 4))

FileNotFoundError: calitp-analytics-data/data-analyses/rt_delay/rt_trips/4_05_04.parquet

In [ ]:
rt_day.stop_delay_view.dropna(subset=['stop_id']) >> head(3)

In [ ]:
corridor = gpd.read_file('./CMCP_StudyAreaBoundary.geojson')

In [ ]:
rt_day.add_corridor(corridor)

# Algorithm

* Clip stops to corridor
* Get first and last stops in corridor for each shape_id, then find 1 stop before and 1 stop after for each
* Filter stop_delay_view based on those stops
* Assert that more than 50% of filtered stops within corridor, if not then drop shape (remove shapes starting/ending in corridor but mostly running outside)

In [ ]:
## TODO add map corr method
mappable_stops = (rt_day.stop_delay_view
                  >> distinct(_.shape_id, _.stop_sequence, _keep_all=True)
                  >> filter(_.corridor)
                  >> select(_.stop_id, _.geometry, _.stop_sequence)
                 )

In [ ]:
# pd.concat([mappable_stops, rt_day.corridor]).explore(tiles = "CartoDB positron")

In [ ]:
with_entry_delay = rt_day.corridor_stop_delays

In [ ]:
## TODO hist. methods if desired

In [ ]:
(with_entry_delay >> filter(_.corridor_delay_seconds < _.corridor_delay_seconds.quantile(.99))).hist(column='corridor_delay_seconds')

In [ ]:
with_entry_delay >> summarize(median_corridor_delay_seconds = _.corridor_delay_seconds.median())

In [ ]:
with_entry_delay >> group_by(_.trip_id) >> summarize(median_corridor_delay_seconds = _.corridor_delay_seconds.median())

In [ ]:
(with_entry_delay >> group_by(_.trip_id) >> summarize(median_corridor_delay_seconds = _.corridor_delay_seconds.median())).hist(column='median_corridor_delay_seconds')

In [ ]:
(with_entry_delay
 >> group_by(_.trip_id)
 >> summarize(median_corridor_delay_seconds = _.corridor_delay_seconds.median())
 >> summarize(sum_of_medians_minutes = _.median_corridor_delay_seconds.sum() / 60)
)

# Report:

* Sum of median trip delays
* Median segment speed
* Delay compared to speed standards: 16mph (approx. 75th %ile for medium size operator by 2010 NTD)

# Speed-based Metrics

In [ ]:
pbar = tqdm()

In [ ]:
importlib.reload(rt_filter_map_plot)

In [ ]:
m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)

In [ ]:
m

In [26]:
may4_metrics = rt_filter_map_plot.RtFilterMapper.corridor_metrics(rt_day)

In [27]:
may4_metrics

{'schedule_metric': 609.7583333333333, 'speed_metric': 5651.118044582289}